# **The Chat Format**

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

## Setup

In [1]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [2]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def get_completion(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message.content


def get_completion_from_messages(message, model="gpt-3.5-turbo", temperature=0): 
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message.content

In [3]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [4]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

Forsooth, the chicken crossed the road to reach the other side and escape the fowl-tempered jester of fate!


In [5]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hello Isa! It's nice to meet you. How can I assist you today?


In [6]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but as a chatbot, I don't have the capability to know your personal information. Can I help you with anything else?


In [7]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Isa!


# OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant. 

In [8]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)


In [11]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [{
    "role": "system",
    "content": """
You are OrderBot, a friendly pizza cashier.

Clarification rule (slot-filling):
- Do NOT ask “what would you like?” or list categories (pizza/sides/drinks).
- Identify what the user already chose, then ask ONLY for missing required details.

Required details:
- For pizza: type (pepperoni/cheese/eggplant) + size + quantity.
- For drinks: type + size + quantity.

Examples:
- If user says “I want pizza”: ask “Which pizza (pepperoni/cheese/eggplant) and what size?”
- If user says “I want a pizza with coke”: ask “Which pizza (pepperoni/cheese/eggplant), what pizza size, and what coke size?”

Vague input rule:
- If the user says something vague like “I want pizza”, ask specifically:
  1) Which pizza (pepperoni/cheese/eggplant)?
  2) What size?
- Do not ask a generic “what would you like?” in that case.

Rules:
- Respond as a friendly cashier in plain English. Do not use JSON formatting.
- DO NOT guess missing details. Ask questions.
- Only use items from the menu.
- Ask one question at a time.
- After order is complete: pickup/delivery (address if delivery), then summary + confirmation, then payment.

Menu:
Pizzas: pepperoni, cheese, eggplant
Toppings: extra cheese, mushrooms, sausage, canadian bacon, AI sauce, peppers
Sides: fries, greek salad
Drinks: coke, sprite, bottled water
"""
}]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'e1c7d15a-f03b-41a2-a454-7f93e7486c75': {'version…

In [10]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

I'm here to help you with your order! Let's start by choosing your items. Would you like to order a pizza, sides, drinks, or anything else from the menu?


## Try experimenting on your own!

You can modify the menu or instructions to create your own orderbot!

# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

### What happened (results)

- The biggest behavioral shift came from the JSON-summary system message. When that message was appended, the assistant consistently returned structured JSON output and often filled in missing order details (pizza type/size/toppings/drink/sides) even when the user didn’t specify them. This created incorrect orders and is effectively a hallucination pattern triggered by forcing a rigid schema too early.

- When I removed the JSON-summary system message, the assistant returned to plain text, but the conversation did not instantly become “cashier correct.” In some runs, the assistant still behaved as if the goal was to summarize or refused JSON directly, implying that either:

- “JSON” language was still present elsewhere in the prompt/messages, or

- Panel was calling an older callback or reusing stale conversation state.

- A separate issue was the UI environment: Panel + VS Code widgets produced loading/spinner and WidgetManager/comm issues. This made experimentation inconsistent because the UI sometimes hung or reused older bindings.

### What I learned

- System messages are extremely powerful: adding a second system instruction (like “create a JSON summary…”) can completely override the intended chatbot behavior.

- Structured outputs (JSON) should be generated only after the order is complete and confirmed—otherwise the model tends to “complete the schema” by guessing.

- In interactive notebook UIs, debugging the message payload and state (what is actually passed to the model) matters as much as prompt wording.